In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

import keras
import keras.backend
from keras import layers
from keras import models
from keras import optimizers
import keras.utils

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sklearn
from sklearn.utils import shuffle

import os
import pickle

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


## Useful Functions

In [2]:
def build_CNN():
    input2 = [layers.Input(shape = (len(X_train[i][0]),)) for i in range(1,len(X_train))]
    input1 = layers.Input(shape = (grid, grid,1))
    x = layers.Conv2D(32, (5, 5), activation = 'relu', padding = 'same')(input1)
    x = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(x)
    x = layers.Conv2D(32, (2, 2), activation = 'relu', padding = 'same')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPool2D((2, 2))(x)
    x1 = layers.Flatten()(x)
    if input2 == []:
        x = layers.Dense(64, activation='relu', name = 'dense1')(x1)
    else:
        x = layers.concatenate(inputs = [x1] + input2, axis = -1)
        x = layers.Dense(64, activation='relu', name = 'dense2')(x)
    x = layers.Dense(128, activation='relu', name = 'dense3')(x)
    output = layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs= [input1] + input2,
                         outputs = output)
    opt = keras.optimizers.Adam(lr = 0.001,
                                beta_1 = 0.9,
                                beta_2 = 0.999,
                                amsgrad = False)
    model.compile(loss = 'binary_crossentropy',
                optimizer = opt,
                metrics = ['binary_crossentropy', 'accuracy'])
    return model

In [3]:
def build_DNN():
    input2 = [layers.Input(shape = (len(X_train[i][0]),)) for i in range(0,len(X_train))]
    if len(input2) == 1:
        x = layers.Dense(64,activation='relu')(input2[0])
    else:
        x = layers.concatenate(inputs = input2, axis = -1)
        x = layers.Dense(64, activation = 'relu')(x)
    x = layers.Dense(128, activation = 'relu')(x)
    output = layers.Dense(2, activation = 'softmax')(x)
    model = models.Model(inputs = input2, 
                         outputs = output)
    opt=keras.optimizers.Adam(lr = 0.0005,
                              beta_1 = 0.9,
                              beta_2 = 0.9,
                              amsgrad = False)
    model.compile(loss = 'binary_crossentropy',
                optimizer = opt,
                metrics = ['binary_crossentropy', 'accuracy'])
    return model

In [4]:
def build_XY(features, label, dic1, dic2):
    X = [np.concatenate((dic1[key], dic2[key])) for key in features]
    Y = [np.concatenate((dic1[key], dic2[key])) for key in label]
    dim = [ele.shape+(1,) for ele in X]
    for i in range(0,len(features)):
        X[i] = X[i].reshape(dim[i])
    return X,Y

## Import data:
Using Train and Test set produced by preprocessing notebook.

In [5]:
grid = 16
train_sig = np.load('/mnt/data/ml/PreProcessing/ShowJets_train_ZZ.npz')
train_bkg = np.load('/mnt/data/ml/PreProcessing/ShowJets_train_QCD.npz')
#train_sig = np.load('../data/ShowJets_train_ZZ.npz')
#train_bkg = np.load('../data/ShowJets_train_QCD.npz')

In [6]:
n_train = len(train_sig['jetPt'])
print(n_train)

276147


In [7]:
removeFeat = ['jetconstPt_log',
 'jetconstEta_abs',
 'jetconstE_log',
 'jetconstPt_Jetlog',
 'charge',
 'isEle',
 'isPho',
 'isMuon',
 'isCh',
 'isNh',
 'delta_eta',
 'delta_phi',
 'deltaR_jet',
 'deltaR_subjet0',
 'deltaR_subjet1',
 'dxy',
 'dz',
 'jetpull',
 'labels']
feat_all = [key for key in train_sig.keys()];
for feat in removeFeat:
    feat_all.remove(feat)
# Swap jetImages into first place for automatized Model building
ind_image = feat_all.index('jetImages')
feat_all[0], feat_all[ind_image] = feat_all[ind_image], feat_all[0]

In [8]:
feat_all

['jetImages',
 'jetMassSD',
 'deltaR_subjets',
 'jetPt',
 'z',
 'jetMass',
 'jetEta',
 'jetPhi',
 'tau1_b05',
 'tau2_b05',
 'tau3_b05',
 'tau1_sd_b05',
 'tau2_sd_b05',
 'tau3_sd_b05',
 'tau1_b10',
 'tau2_b10',
 'tau3_b10',
 'tau1_sd_b10',
 'tau2_sd_b10',
 'tau3_sd_b10',
 'tau1_b15',
 'tau2_b15',
 'tau3_b15',
 'tau1_sd_b15',
 'tau2_sd_b15',
 'tau3_sd_b15',
 'tau1_b20',
 'tau2_b20',
 'tau3_b20',
 'tau1_sd_b20',
 'tau2_sd_b20',
 'tau3_sd_b20',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'beta3',
 'beta3_sd',
 'tau21',
 'jetpull_abs',
 'dxy_max',
 'dz_max']

# Build CNN

## Build Model with only Images

In [9]:
# features will be loaded into X
features = ['jetImages']

# label into Y
label = ['labels']
X_train, Y_train = build_XY(features, label, train_sig, train_bkg)
Y_train = [Y_train[0][:,:2]]

In [10]:
CNN = build_CNN()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 4ms/step
[[0.50025594 0.49974406]
 [0.49959326 0.50040674]
 [0.49986622 0.5001338 ]
 [0.4997814  0.50021863]
 [0.50004095 0.49995908]
 [0.50019866 0.4998013 ]
 [0.500176   0.49982402]
 [0.5000398  0.49996015]
 [0.49949938 0.50050056]
 [0.49961916 0.5003809 ]]
[0.6929615139961243, 0.6929615139961243, 0.5]


In [12]:
checkpoint_path = "model/CNN_Images.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS, 
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 441835 samples, validate on 110459 samples
Epoch 1/80
441835/441835 [==============================] - 240s 544us/step - loss: 0.5579 - binary_crossentropy: 0.5579 - acc: 0.6988 - val_loss: 0.6387 - val_binary_crossentropy: 0.6387 - val_acc: 0.6899

Epoch 00001: val_loss improved from inf to 0.63871, saving model to model/CNN_Images.h1
Epoch 2/80
441835/441835 [==============================] - 241s 544us/step - loss: 0.5361 - binary_crossentropy: 0.5361 - acc: 0.7145 - val_loss: 0.7659 - val_binary_crossentropy: 0.7659 - val_acc: 0.6482

Epoch 00002: val_loss did not improve from 0.63871
Epoch 3/80
441835/441835 [==============================] - 239s 542us/step - loss: 0.5305 - binary_crossentropy: 0.5305 - acc: 0.7189 - val_loss: 0.6911 - val_binary_crossentropy: 0.6911 - val_acc: 0.6475

Epoch 00003: val_loss did not improve from 0.63871
Epoch 4/80
441835/441835 [==============================] - 240s 544us/step - loss: 0.525

In [13]:
if not os.path.exists("history"):
    os.mkdir("history")
with open('history/CNN_Images.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

## Build Model with all Features

### Select Features for Model

Show all possible features

In [14]:
# features will be loaded into X
features = feat_all

# label into Y
label = ['labels']

In [15]:
X_train, Y_train = build_XY(features, label, train_sig, train_bkg)
Y_train = [Y_train[0][:,:2]]

Initialize model:

In [16]:
CNN = build_CNN()

Test model's prediction $before$ training:

In [17]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 9ms/step
[[0.5061053  0.4938947 ]
 [0.4979751  0.5020249 ]
 [0.49662575 0.50337416]
 [0.4896791  0.5103209 ]
 [0.4931998  0.5068002 ]
 [0.49717    0.50283   ]
 [0.49570665 0.5042933 ]
 [0.49475947 0.50524056]
 [0.500486   0.49951392]
 [0.5027973  0.49720272]]
[0.6881012320518494, 0.6881012320518494, 0.699999988079071]


Now train! (warning: if building CNN, computer tends to get loud)

In [18]:
checkpoint_path = "model/CNN_all.h8"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 80
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Train on 441835 samples, validate on 110459 samples
Epoch 1/80
441835/441835 [==============================] - 259s 587us/step - loss: 0.3622 - binary_crossentropy: 0.3622 - acc: 0.8319 - val_loss: 0.3764 - val_binary_crossentropy: 0.3764 - val_acc: 0.8632

Epoch 00001: val_loss improved from inf to 0.37639, saving model to model/CNN_all.h8
Epoch 2/80
441835/441835 [==============================] - 259s 586us/step - loss: 0.3330 - binary_crossentropy: 0.3330 - acc: 0.8480 - val_loss: 0.3802 - val_binary_crossentropy: 0.3802 - val_acc: 0.8670

Epoch 00002: val_loss did not improve from 0.37639
Epoch 3/80
441835/441835 [==============================] - 257s 582us/step - loss: 0.3267 - binary_crossentropy: 0.3267 - acc: 0.8510 - val_loss: 0.3868 - val_binary_crossentropy: 0.3868 - val_acc: 0.8521

Epoch 00003: val_loss did not improve from 0.37639
Epoch 4/80
441835/441835 [==============================] - 261s 591us/step - loss: 0.3228 - binary_crossentropy: 0.3228 - acc: 0.8528 - val

In [19]:
with open('history/CNN_all.h8.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

### Select Features for Model

Show all possible features

In [20]:
feat_all

['jetImages',
 'jetMassSD',
 'deltaR_subjets',
 'jetPt',
 'z',
 'jetMass',
 'jetEta',
 'jetPhi',
 'tau1_b05',
 'tau2_b05',
 'tau3_b05',
 'tau1_sd_b05',
 'tau2_sd_b05',
 'tau3_sd_b05',
 'tau1_b10',
 'tau2_b10',
 'tau3_b10',
 'tau1_sd_b10',
 'tau2_sd_b10',
 'tau3_sd_b10',
 'tau1_b15',
 'tau2_b15',
 'tau3_b15',
 'tau1_sd_b15',
 'tau2_sd_b15',
 'tau3_sd_b15',
 'tau1_b20',
 'tau2_b20',
 'tau3_b20',
 'tau1_sd_b20',
 'tau2_sd_b20',
 'tau3_sd_b20',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'beta3',
 'beta3_sd',
 'tau21',
 'jetpull_abs',
 'dxy_max',
 'dz_max']

In [21]:
drop_those = [ 'jetImages',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult']
# features will be loaded into X
features = feat_all[:]
for feat in drop_those:
    features.remove(feat)
# label into Y
label = ['labels']

In [22]:
features

['jetMassSD',
 'deltaR_subjets',
 'jetPt',
 'z',
 'jetMass',
 'jetEta',
 'jetPhi',
 'tau1_b05',
 'tau2_b05',
 'tau3_b05',
 'tau1_sd_b05',
 'tau2_sd_b05',
 'tau3_sd_b05',
 'tau1_b10',
 'tau2_b10',
 'tau3_b10',
 'tau1_sd_b10',
 'tau2_sd_b10',
 'tau3_sd_b10',
 'tau1_b15',
 'tau2_b15',
 'tau3_b15',
 'tau1_sd_b15',
 'tau2_sd_b15',
 'tau3_sd_b15',
 'tau1_b20',
 'tau2_b20',
 'tau3_b20',
 'tau1_sd_b20',
 'tau2_sd_b20',
 'tau3_sd_b20',
 'beta3',
 'beta3_sd',
 'tau21',
 'jetpull_abs',
 'dxy_max',
 'dz_max']

In [23]:
#X_train, Y_train = build_XY(features,label,data_train)
X_train, Y_train = build_XY(features, label, train_sig, train_bkg)
Y_train = [Y_train[0][:,:2]]

Initialize model:

In [24]:
CNN = build_DNN()

Test model's prediction $before$ training:

In [25]:
X_batch = [ele[:10] for ele in X_train]
Y_batch = [ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y = Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 7ms/step
[[0.5486917  0.45130834]
 [0.6293871  0.37061295]
 [0.6313691  0.3686309 ]
 [0.55587584 0.4441242 ]
 [0.6003828  0.3996172 ]
 [0.57158464 0.4284154 ]
 [0.55627143 0.4437286 ]
 [0.54628944 0.4537106 ]
 [0.61265916 0.38734087]
 [0.5552818  0.44471818]]
[0.8724324107170105, 0.8724324107170105, 0.0]


Now train! (warning: if building CNN, computer tends to get loud)

In [26]:
checkpoint_path = "model/DNN_noflavour.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', 
                                                   verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', 
                                                   period = 1)    
EPOCHS = 60
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
history = CNN.fit(
    X_train, Y_train,
    epochs = EPOCHS,
    validation_split = 0.2,
    verbose = 1,
    callbacks = [early_stop, model_checkpoint])

Train on 441835 samples, validate on 110459 samples
Epoch 1/60
441835/441835 [==============================] - 39s 87us/step - loss: 0.3845 - binary_crossentropy: 0.3845 - acc: 0.8201 - val_loss: 0.3562 - val_binary_crossentropy: 0.3562 - val_acc: 0.8847

Epoch 00001: val_loss improved from inf to 0.35617, saving model to model/DNN_noflavour.h1
Epoch 2/60
441835/441835 [==============================] - 38s 87us/step - loss: 0.3470 - binary_crossentropy: 0.3470 - acc: 0.8407 - val_loss: 0.4804 - val_binary_crossentropy: 0.4804 - val_acc: 0.8118

Epoch 00002: val_loss did not improve from 0.35617
Epoch 3/60
441835/441835 [==============================] - 39s 87us/step - loss: 0.3411 - binary_crossentropy: 0.3411 - acc: 0.8439 - val_loss: 0.5720 - val_binary_crossentropy: 0.5720 - val_acc: 0.7322

Epoch 00003: val_loss did not improve from 0.35617
Epoch 4/60
441835/441835 [==============================] - 38s 86us/step - loss: 0.3409 - binary_crossentropy: 0.3409 - acc: 0.8450 - val_l

In [27]:
with open('history/DNN_noflavour.h1.history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)